<a href="https://colab.research.google.com/github/fishfishin/CT-denoise/blob/main/sigma_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
#import visdom
import torchvision.transforms as T
import skimage.external.tifffile
import copy
import cv2
import math
from skimage import filters
from torch.autograd import Variable
from math import exp

from skimage.transform import radon,iradon
from skimage.restoration import denoise_nl_means
from google.colab import drive
"""
import tensorflow as tf
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
"""
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
print(dev)
#vis = visdom.Visdom()

class doubleDQN(nn.Module):

    def __init__(self, h, w, output1,output2):
        super(doubleDQN, self).__init__()

        # image patch size is   h x w
        #first two  shared conv layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=(2),padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=(2),padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        # patch size after conv layer
        def conv2d_size_out(size, kernel_size = 3, stride = 2):
            return size // stride  + 1
        convw = conv2d_size_out(conv2d_size_out(w))
        convh = conv2d_size_out(conv2d_size_out(h))

        linear_input_size = convw * convh * 64

        #  Parameter selection
        #  conv 64 Filter, kernel 3x3 
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1,padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        #  FCL 128 neurons
        self.dense1 = nn.Linear(linear_input_size, 128)
        self.bn4 = nn.BatchNorm1d(128)
        self.head1 = nn.Linear(128,output1)
       
        linear_input_size = convw * convh  * 128
        #  Parameter tuning
        #  conv 128 Filter, kernel 3x3 
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1,padding=1)
        self.bn5 = nn.BatchNorm2d(128)
        #  conv 128 Filter, kernel 3x3 
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, stride=1,padding=1)
        self.bn6 = nn.BatchNorm2d(128)
        #  FCL 256 neurons
        self.dense2 = nn.Linear(linear_input_size, 256)
        self.bn7 = nn.BatchNorm1d(256)
        self.head2 = nn.Linear(256, output2)

        self.opt = optim.SGD(self.parameters(), lr=0.001, momentum=0.9)

    
    def forward(self, x):


        # 2 shared layers
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))

        #  Parameter selection
        out1 = F.relu(self.bn3(self.conv3(x)))
        out1 = out1.view(out1.size(0), -1)
        out1 = F.relu(self.bn4(self.dense1(out1)))
        out1 = self.head1(out1)
       
        #  Parameter tuning
        out2 = F.relu(self.bn5(self.conv4(x)))
        out2 = F.relu(self.bn6(self.conv5(out2)))
        out2 = out2.view(out2.size(0), -1)
        out2 = F.relu(self.bn7(self.dense2(out2)))
        out2 = self.head2(out2)

        return  out1, out2


    def update(self, x,target1,target2):
        #  inputs, labels = inputs.to(device), labels.to(device)
        x = x.cuda()
        out1,out2 = self.forward(x)
        self.opt.zero_grad()
        
        loss1 = nn.MSELoss()
        loss2 = nn.MSELoss()
        upd_1 = loss1(out1,target1 )
        upd_2 = loss2(out2,target2 )
        upd_ = upd_1 + upd_2
        upd_.backward()
        
        self.opt.step()

        return upd_


####  1st path
Para = [ 'sigma']

####  2nd path
Actions = ['+1.5','1.1','null','0.9','-1.5']

MAX_EPOCHS = 20
DISCOUNT_RATE = 0.99
RESOLUTION = 256
PATCH_SIZE = [9,9]
Patch_num = RESOLUTION **2
PATCH_reward = 5
TARGET_UPDATE_STEP = 300
MAXSTEPS_FILTER= 30
REPLAY_MEMORY = 100000  ######### buffer？
BATCH_SIZE = 128

TUNING_STEP= 20

def replay_train(mainDQN: doubleDQN, targetDQN: doubleDQN, states, next_states, action,parameter, rewards) -> np.float64:

    X = torch.zeros(states.shape[0], 1, PATCH_SIZE[0],PATCH_SIZE[0], dtype=torch.float64)
    X1 = torch.zeros(states.shape[0], 1, PATCH_SIZE[0],PATCH_SIZE[0], dtype=torch.float64)
    X[:,0,:,:] = torch.from_numpy(states)
    X1[:,0,:,:] = torch.from_numpy(next_states)

    t1, t2 = targetDQN(Variable(X1).cuda())  #### old version of NET
    y1, y2 = mainDQN(Variable(X).cuda())

    ####  1st path
    tem = torch.max(t1, axis=1)
    temp = tem.values
    Q_target1 = rewards + temp.cpu().detach().numpy() *DISCOUNT_RATE
    for i in range(y1.shape[0]):
        y1[i,int(parameter[i])] = Q_target1[i]

    ####  2nd path
    tem = torch.max(t2, axis=1)
    temp = tem.values
    Q_target2 = rewards + temp.cpu().detach().numpy() *DISCOUNT_RATE
    for i in range(y2.shape[0]):
        y2[i,int(action[i])] = Q_target2[i]

    

    return mainDQN.update(X, y1,y2)

def PSNR(img1, img2):
    D = np.array(img1 - img2, dtype=np.float64)
    D[:, :] = D[:, :]**2
    RMSE = D.sum()/img1.size
    psnr = 10*math.log10(float(1.0**2)/RMSE)
    return psnr


def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()

def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = Variable(_2D_window.expand(channel, 1, window_size, window_size).contiguous())
    return window

def _ssim(img1, img2, window, window_size, channel, size_average = True):
    mu1 = F.conv2d(img1, window, padding = window_size//2, groups = channel)
    mu2 = F.conv2d(img2, window, padding = window_size//2, groups = channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1*mu2

    sigma1_sq = F.conv2d(img1*img1, window, padding = window_size//2, groups = channel) - mu1_sq
    sigma2_sq = F.conv2d(img2*img2, window, padding = window_size//2, groups = channel) - mu2_sq
    sigma12 = F.conv2d(img1*img2, window, padding = window_size//2, groups = channel) - mu1_mu2

    C1 = 0.01**2
    C2 = 0.03**2

    ssim_map = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)


def ssim(img1, img2, window_size = 11, size_average = True):
    (_, channel, _, _) = img1.size()
    #channel = 1
    window = create_window(window_size, channel)
    
    if img1.is_cuda:
        window = window.cuda(img1.get_device())
    window = window.type_as(img1)
    
    return _ssim(img1, img2, window, window_size, channel, size_average)

def divide_patch(fimg) -> np.float64:

    fimgpad = np.zeros((RESOLUTION + PATCH_SIZE[0] -1, RESOLUTION + PATCH_SIZE[0] -1),dtype =np.float64)
    fimgpad[int((PATCH_SIZE[0]+1)/2)-1:RESOLUTION+int((PATCH_SIZE[0]+1)/2)-1,int((PATCH_SIZE[0]+1)/2)-1:RESOLUTION+int((PATCH_SIZE[0]+1)/2)-1]=fimg
    state = np.zeros((Patch_num,PATCH_SIZE[0], PATCH_SIZE[0]))
    count = 0
    for xcord in range(RESOLUTION):
        for ycord in range(RESOLUTION):
            temp=fimgpad[xcord:xcord+PATCH_SIZE[0],ycord:ycord+PATCH_SIZE[0]]
            state[count,:,: ] = temp
            count += 1
    return state


def Denoise(state, parameter, action, parameter_value, GroundTruth, original_image) -> np.float64:

    current_patch = np.zeros((Patch_num, PATCH_SIZE[0], PATCH_SIZE[0]),dtype=np.float64)
    org_patch = divide_patch(original_image)
    # tuning parameter
    #d = torch.zeros((1,1,9,9),dtype=torch.float64)
    for idx in range(Patch_num):
        if parameter[idx] ==0:
            if action[idx]==0:
                parameter_value[idx,0] = parameter_value[idx,0] *1.5
            if action[idx]==1:
                parameter_value[idx,0] = parameter_value[idx,0] *1.1
            if action[idx]==3:
                parameter_value[idx,0] = parameter_value[idx,0]*0.9
            if action[idx]==4:
                parameter_value[idx,0] = parameter_value[idx,0] *0.5
          
            
        """
        if parameter[idx]==2:
            if action[idx]==0:
                parameter_value[idx,2] = parameter_value[idx,1] *1.5
            if action[idx]==1:
                parameter_value[idx,2] = parameter_value[idx,1] *1.1
            if action[idx]==3:
                parameter_value[idx,2] = parameter_value[idx,1]*0.9
            if action[idx]==4:
                parameter_value[idx,2] = parameter_value[idx,1] *0.5
        """
        #model = NLM(parameter_value[idx,0], parameter_value[idx,1]).cuda()
        #print(" finish assignment")
        
        #d[:,:,:,:] = torch.from_numpy(org_patch[idx,:,:])
        final_patch = filters.gaussian(org_patch[idx,:,:],parameter_value[idx,0])# denoise_nl_means(org_patch[idx,:,:], h = parameter_value[idx,0],fast_mode=True,patch_size=1,patch_distance=3,multichannel=False)
        current_patch [idx, :, :] = final_patch
        #print("patch: {}".format(idx))
    print("strat rewarding")
    ############# NLM

    ### how to stitch  256x 256 patches into a single image fimg  after Bm3d
    next_img = np.reshape(current_patch[:, int(PATCH_SIZE[0]//2), int(PATCH_SIZE[0]//2)], (RESOLUTION, RESOLUTION), order='A')
    next_state = current_patch
    current_image = np.reshape(state[:, int(PATCH_SIZE[0]//2), int(PATCH_SIZE[0]//2)], (RESOLUTION, RESOLUTION), order='A')
    #############   calculate reward and error

    dist1img = current_image - GroundTruth
    dist2img = next_img - GroundTruth  #######################################
    dist2 = np.reshape(dist2img, (Patch_num), order='A')

    dist1imgLarge = np.zeros((RESOLUTION+PATCH_reward-1,RESOLUTION+PATCH_reward-1))
    margin = int((PATCH_reward-1)/2)
    dist1imgLarge[margin:RESOLUTION+margin,margin:RESOLUTION+margin]=np.absolute(dist1img)

    dist2imgLarge = np.zeros((RESOLUTION + PATCH_reward-1, RESOLUTION + PATCH_reward-1))
    dist2imgLarge[margin:RESOLUTION + margin, margin:RESOLUTION + margin] = np.absolute(dist2img)

    rewardimg = np.zeros((RESOLUTION,RESOLUTION))
    
    for i in range(RESOLUTION):
        for j in range(RESOLUTION):
           
            rewardimg[i,j]= 1/(np.sum(dist2imgLarge[i:i+PATCH_reward,j:j+PATCH_reward])+0.001) - 1/(np.sum(dist1imgLarge[i:i+PATCH_reward,j:j+PATCH_reward])+0.001)
    reward = np.reshape(rewardimg,(Patch_num),order='A')
    error = np.sum(np.absolute(dist2))


    return next_state, reward, parameter_value, next_img ,error


def main():


    drive.mount('/content/drive')
    TrueImgTrain = skimage.external.tifffile.imread("drive/My Drive/Colab Notebooks/G.tif")/4095.0
    datasize = TrueImgTrain.shape
    theta = np.linspace(0., 180., max(TrueImgTrain.shape), endpoint=False)
    sinogram = np.zeros((16,256,256))
    for i in range(datasize[0]): sinogram[i,:,:] = radon(TrueImgTrain[i,:,:], theta=theta, circle=True)
    
    

    # (None, channel, H, w, depth) for Volume
    mainDQN = doubleDQN(PATCH_SIZE[0], PATCH_SIZE[1], len(Para),len(Actions)).cuda().double()
    targetDQN = doubleDQN(PATCH_SIZE[0], PATCH_SIZE[1], len(Para),len(Actions)).cuda().double()

    state_sel = np.zeros((REPLAY_MEMORY, PATCH_SIZE[0], PATCH_SIZE[0]))
    next_state_sel = np.zeros((REPLAY_MEMORY, PATCH_SIZE[0], PATCH_SIZE[0]))
    action_sel = np.zeros((REPLAY_MEMORY))
    reward_sel = np.zeros((REPLAY_MEMORY))
    para_sel = np.zeros((REPLAY_MEMORY))
    
    indicator = 0 
    if MAX_EPOCHS>0:  ########################## 
                   
            State = np.zeros((datasize[0], Patch_num, PATCH_SIZE[0],PATCH_SIZE[0]))             ##  slabs  x  256**2 x 9 x 9
            parameter_value = 0.99 *np.ones((Patch_num, len(Para)), dtype=np.float64)
            
            #sigma = np.random.rand(2)*10
            #gaussian = np.random.normal(sigma[0], sigma[1], (datasize[0],datasize[1],datasize[2]))/4095.0
            noise_mask = np.random.poisson(TrueImgTrain)
            train_data = np.zeros((16,256,256))
            for i in range(datasize[0]):   train_data[i,:,:] = iradon(sinogram[i,:,:] + noise_mask[i,:,:], theta=theta, circle=True)

             
            # for each image initializer
            for IMG in range(datasize[0]):
                state = divide_patch(  train_data[ IMG,:,:]  )  ##  256**2 x 9 x 9              
                GroundTruth = TrueImgTrain[IMG,:, : ]      
                ## initialize the 1st and 2nd paths 
                parameter = np.ones((Patch_num))
                action = 2 * np.ones((Patch_num))
                next_state, reward, parameter_value, img, error = Denoise( state,  parameter, action, parameter_value , GroundTruth, train_data[ IMG,:,:] )
                State[ IMG, :, :, :] = next_state
                
                print(IMG)


            State_initial = State
            count_memory = 0

            for episode in range(MAX_EPOCHS-1):

                e = 0.999 / ((episode / 150) + 1)
                if e<0.1:
                    e=0.1
                step_count = 0
                State = State_initial

                for ITER_NUM in range(MAXSTEPS_FILTER):

                    for IMG_IDX in range(datasize[0]):
                       
                        state = State[ IMG_IDX, :, :, :]     ##  slabs x num of patches x 9 x 9
                        GroundTruth = TrueImgTrain[IMG_IDX,:, : ]                      
                        parameter = np.ones((Patch_num))
                        action = 2 * np.ones((Patch_num))

                        # random select patches and     action  for each image
                        flag = np.random.rand(Patch_num)
                        count_patch = 0
                        length_patch = 0
                        for idx in range(Patch_num):
                            if flag[idx]>=e:
                                length_patch += 1
                        
                        # yy  : patch samples
                        print("start sampling patches")
                        yy = torch.zeros(length_patch, 1, PATCH_SIZE[0],PATCH_SIZE[0], dtype=torch.float64)
                        for idx in range(Patch_num):
                            if flag[idx]<e:
                                action[idx] = np.random.randint(len(Actions), size=1)
                                parameter[idx] = np.random.randint(len(Para), size=1)
                            if flag[idx]>=e:
                                yy[count_patch,0,:, :] = torch.from_numpy(state[idx,:,:])
                                count_patch += 1
                        
                        y1, y2 = mainDQN(Variable(yy).cuda())
                        parameter_yy = torch.argmax(y1, axis=1)
                        action_yy = torch.argmax(y2, axis=1)
                     
                        #### action and paramter chosen
                        count_patch=0
                        for idx in range(Patch_num):
                            if flag[idx] >= e:
                                action[idx] = action_yy[count_patch]
                                parameter[idx] = parameter_yy[count_patch]
                                count_patch += 1
                        print("strat denoise")
                        next_state, reward, parameter_value, img, error = Denoise( state,  parameter, action,parameter_value,  GroundTruth, train_data[ IMG_IDX,:,:] )
                        psnr =PSNR(GroundTruth, img )
                        
                        img1 = torch.zeros(1, 1, datasize[1],datasize[2],dtype=torch.float64)
                        img2 = torch.zeros(1, 1, datasize[1],datasize[2],dtype=torch.float64)
                        img1[:,0,:,:] = torch.from_numpy(GroundTruth)
                        img2[:,0,:,:] = torch.from_numpy(img)
                        ssim_ = ssim(img1,img2)
                        
                        print(" current PSNR : {} ssim : {} ".format(psnr,ssim_))
                        name = str(step_count)
                        #vis.image( img,opts='store_history')
                        
                        ###################  ? random replacement
                        sel_prob = 0.01
                        flag1 = np.random.rand(Patch_num)
                        flag2 = np.zeros([Patch_num])
                        for idx in range(Patch_num):
                            if flag1[idx]>=sel_prob:
                                flag2[idx] = 0
                            if flag1[idx]<sel_prob:  ##### chosen
                                flag2[idx] = 1

                        sel_num = int(np.sum(flag2))

                        ##### refresh the buffer randomly
                        if count_memory+sel_num<=REPLAY_MEMORY-2:
                            for idx in range(Patch_num):
                                if flag1[idx]<sel_prob:
                                    state_sel[count_memory,:,:] = state[idx,:,:]
                                    next_state_sel[count_memory,:,:] = next_state[idx,:,:]
                                    action_sel[count_memory]=action[idx]
                                    para_sel[count_memory] = parameter[idx]
                                    reward_sel[count_memory] = reward[idx]
                                    #value_sel[count_memory] = parameter_value[idx,:]
                                    
                                    count_memory += 1
                        else:
                            indicator = 1
                            for idx in range(Patch_num):
                                if flag1[idx]<sel_prob:
                                    state_sel[count_memory,:] = state[idx,:,:]
                                    next_state_sel[count_memory,:] = next_state[idx,:,:]
                                    action_sel[count_memory]=action[idx]
                                    para_sel[count_memory] = parameter[idx]
                                    reward_sel[count_memory] = reward[idx]
                                    #value_sel[count_memory] = parameter_value[idx,:]
                                    
                                    if count_memory == REPLAY_MEMORY - 1:
                                        count_memory = 0
                                        print('Replay Memory is full')
                                    else:
                                        count_memory += 1
                        if indicator == 0:
                            replay_size = count_memory +  1
                        else:
                            replay_size = REPLAY_MEMORY

                        if replay_size > BATCH_SIZE:

                            print(" tuning")
                            
                            for i in range(TUNING_STEP):
                                shuffle_order = np.arange(replay_size)
                                np.random.shuffle(shuffle_order)
                                minibatch_state = state_sel[shuffle_order[0:BATCH_SIZE], :, :]
                                minibatch_next_state = next_state_sel[shuffle_order[0:BATCH_SIZE],:,:]
                                minibatch_action = action_sel[shuffle_order[0:BATCH_SIZE]]
                                minibatch_parameter = para_sel[shuffle_order[0:BATCH_SIZE]]
                                minibatch_reward = reward_sel[shuffle_order[0:BATCH_SIZE]]
                                
                                loss = replay_train(mainDQN, targetDQN, minibatch_state,minibatch_next_state,minibatch_action,minibatch_parameter, minibatch_reward)
                                if step_count % TARGET_UPDATE_STEP == 0:
                                    targetDQN = copy.deepcopy(mainDQN)
                                step_count += 1
                                #print(step_count)

                        State[IMG_IDX, :, :, :] = next_state
                        

                    print("Episode: {}  Iterations: {} Loss: {}".format(episode, ITER_NUM, loss))

                CHECK = episode+1
    # save the trained networks
    PATH = "drive/My Drive/Colab Notebooks/mainDQN"
    torch.save(mainDQN.state_dict(), PATH)
    PATH = "drive/My Drive/Colab Notebooks/targetDQN"
    torch.save(targetDQN.state_dict(), PATH)

if __name__ == '__main__':
    main()

cuda:0
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/skimage/transform/radon_transform.py:91: UserWarning: Radon transform: image must be zero outside the reconstruction circle
  warn('Radon transform: image must be zero outside the '


Streaming output truncated to the last 5000 lines.
strat denoise
strat rewarding
 current PSNR : 30.545852820513847 ssim : 0.791419522325955 
 tuning
start sampling patches
strat denoise
strat rewarding
 current PSNR : 30.565802117151264 ssim : 0.7904083232744427 
 tuning
start sampling patches
strat denoise
strat rewarding
 current PSNR : 30.639699710024537 ssim : 0.7912899074885973 
 tuning
start sampling patches
strat denoise
strat rewarding
 current PSNR : 30.60912784721274 ssim : 0.7880221848636755 
 tuning
Episode: 10  Iterations: 9 Loss: 3.7505137455185466e-05
start sampling patches
strat denoise
strat rewarding
 current PSNR : 30.57867974519223 ssim : 0.7887700335452216 
 tuning
start sampling patches
strat denoise
strat rewarding
 current PSNR : 30.56252210541902 ssim : 0.7898956283097105 
 tuning
start sampling patches
strat denoise
strat rewarding
 current PSNR : 30.560090562197107 ssim : 0.7903760560699031 
 tuning
start sampling patches
strat denoise
strat rewarding
 curre